In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

from scipy import stats
from scipy.stats import norm, skew

from sklearn.preprocessing import LabelEncoder,StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier, OrthogonalMatchingPursuit


from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
import gc


import sys, os
import random 

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
    
from IPython import display, utils

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 300)



def set_seed(seed=4242):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
set_seed()

### Memory Reduction

In [ ]:
data = pd.read_csv('/kaggle/input/uplift-modeling/criteo-uplift-v2.1.csv',
                   dtype={'f0': 'float32',
                          'f1': 'float32',
                          'f2': 'float32',
                          'f3': 'float32',
                          'f4': 'float32',
                          'f5':'float32',
                          'f6': 'float32',
                          'f7': 'float32',
                          'f8': 'float32',
                          'f9': 'float32',
                          'f10': 'float32',
                          'f11': 'float32',
                          'treatment': 'int8',
                          'conversion': 'int8',
                          'visit': 'int8',
                          'exposure':'int8'})
                          # ,nrows=10000000)


In [ ]:
data.shape

In [ ]:

data.head(20)

In [ ]:
data.isnull().sum()

In [ ]:
from scipy.stats import norm
feats = ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11']
def plot_numerical(feature):
    sns.set()
    plt.style.use('fivethirtyeight')
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
    sns.distplot(data[feature], ax=axes[0], label='train', color='k', kde_kws={"bw":0.01},bins = 40, fit=norm)
    sns.distplot(np.log1p(data[feature]),  ax=axes[1], color="darkcyan",kde_kws={"bw":0.01}, bins=40, fit=norm)
    
    axes[0].set_title('train distribution')
    axes[1].set_title('log1p')
    
    
   


In [ ]:
for c in feats:
    plot_numerical(c)    

In [ ]:
cat_cols = [c for c in data.columns if data[c].dtypes =='int8']

def analyse_cats(df, cat_cols):
    
    for c in cat_cols:
                      
        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
        sns.countplot(df[c], ax=axes[0], label='data counts', palette='bone');
        sns.distplot(df[c],  ax=axes[1], kde_kws={"bw":0.01},color="darkcyan");


        fig.suptitle(c, fontsize=18);
        axes[0].set_title('data');
        print(c ,': ', df[c].value_counts())
        
analyse_cats(data, cat_cols)


In [ ]:
import matplotlib.style as style
style.use('ggplot')
sns.set_style('whitegrid')
plt.subplots(figsize = (20,15))
## Plotting heatmap. 

# Generate a mask for the upper triangle (taken from seaborn example gallery)
mask = np.zeros_like(data.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True


sns.heatmap(data.corr(), cmap=sns.diverging_palette(20, 220, n=200), annot=True, mask=mask, center = 0, );
## Give title. 
plt.title("Heatmap of all the Features", fontsize = 25);

In [ ]:
plt.style.use('fivethirtyeight')
sns.clustermap(data.corr(), center=0, cmap="vlag", method='single', linewidths=.75, figsize=(14, 12))  

In [ ]:
plt.figure(figsize=(7, 4))
sns.countplot(data.conversion)

### Build unseen data

In [ ]:
target = data.pop('conversion')

train, test, tar_train, tar_test = train_test_split(data, target, test_size=0.2)
train.shape, test.shape, tar_train.shape, tar_test.shape

### HistgradientBoosting

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, roc_auc_score
import gc


### HistGradientBoosting

In [ ]:
n_splits = 3
kf = StratifiedKFold(n_splits=n_splits, random_state=2020)
train_oof = np.zeros((train.shape[0],))
test_preds = 0
train_oof.shape

for fold_, (train_index, val_index) in enumerate(kf.split(train, tar_train)):
    print("Fitting fold", fold_)
    train_features = train.iloc[train_index]
    train_target = target.iloc[train_index]
    
    val_features = train.iloc[val_index]
    val_target = target.iloc[val_index]
    
    model = HistGradientBoostingClassifier(max_iter=4000, learning_rate=0.01)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred[:,1]
    print("Fold AUC:", roc_auc_score(val_target, val_pred[:,1]))
    test_preds += model.predict_proba(test)[:,1]/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
oof_r = np.where(train_oof > 0.5, 1, 0)
cf_matrix = confusion_matrix(tar_train, (oof_r)) 
group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ["{0:0.0f}".format(value) for value in
                cf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.4)
plt.style.use('seaborn-poster')
sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='vlag')